In [2]:
#pip install scikit-learn
pip install "mne>=1.0" matplotlib mne-qt-browser

SyntaxError: invalid syntax (225514086.py, line 2)

In [22]:
del EEGDataProcessor

In [3]:

import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

#from matplotlib.colors import TwoSlopeNorm
import math 
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.stats import permutation_cluster_1samp_test as pcluster_test
from mne.time_frequency import tfr_multitaper, tfr_morlet
from mne.time_frequency import morlet, fwhm
import tables as tb
from csv import writer 
import matplotlib
matplotlib.use('Qt5Agg')  # Use the Qt5 backend

%matplotlib

class EEGDataProcessor:

    
    
    def __init__(self):
        self.raw = None
        self.epochs = None 
        self.clean_epochs = None
        self.ICA = None
        
    def load_epochs_from_fif(self, path_to_epochs_file):
        """Load preprocessed epochs from a .fif file."""
        # Load epochs from the .fif file
        self.epochs = mne.read_epochs(path_to_epochs_file)
        

    def read_file(self, path):
        r = mne.io.read_raw_bdf(path, preload=False)
        
        # Define signal info
        n_time_samps = r.n_times
        time_secs = r.times
        ch_names = r.ch_names
        n_chan = len(ch_names) 
        
        
        channels_to_keep = ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10",
                "A11", "A12", "A13", "A14", "A15", "A16", "A17", "A18", "A19", "A20",
                "A21", "A22", "A23", "A24", "A25", "A26", "A27", "A28", "A29", "A30",
                "A31", "A32", "B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B9", "B10",
                "B11", "B12", "B13", "B14", "B15", "B16", "B17", "B18", "B19", "B20",
                "B21", "B22", "B23", "B24", "B25", "B26", "B27", "B28", "B29", "B30", "B31", "B32"]

        channels_to_exclude_str = [ch for ch in r.ch_names if ch not in channels_to_keep + ['Status']]
        
        
        raw = mne.io.read_raw_bdf(path, exclude = channels_to_exclude_str ,  preload=True)
        
        list_chan_name = list_chan_name = { 'A3': 'AF3',
        'A7': 'F7', 'A6': 'F5','A5': 'F3', 'A4': 'F1',
        'A8': 'FT7', 'A9': 'FC5','A10': 'FC3', 'A11': 'FC1',
        'A15': 'T7', 'A14': 'C5','A13': 'C3', 'A12': 'C1',
        'A16': 'TP7', 'A17': 'CP5','A18': 'CP3', 'A19': 'CP1', 'A32': 'CPz',
        'A24': 'P9', 'A23': 'P7','A22': 'P5', 'A21': 'P3', 'A20': 'P1','A31': 'Pz',
        'A25': 'PO7', 'A26': 'PO3','A30': 'POz', 
        'A27': 'O1',   'A29': 'Oz', 
        'A28': 'Iz', 
        'B1': 'Fpz','B2': 'Fp2',
        'B5': 'AFz', 'B4': 'AF4','B3': 'AF8',
        'B6': 'Fz', 'B7': 'F2','B8': 'F4', 'B9': 'F6','B10': 'F8',
        'B15': 'FCz', 'B14': 'FC2','B13': 'FC4', 'B12': 'FC6','B11': 'FT8',
        'B16': 'Cz', 'B17': 'C2','B18': 'C4', 'B19': 'C6','B20': 'T8',
        'B24': 'CP2', 'B23': 'CP4','B22': 'CP6', 'B21': 'TP8',
        'B25': 'P2', 'B26': 'P4','B27': 'P6', 'B28': 'P8','B29': 'P10',
        'B31': 'PO4', 'B30': 'PO8',
        'B32': 'O2'
        }
            
        _ = raw.rename_channels(list_chan_name)
        print(raw.info)
        print("These are the channels and lenght of channnels,", raw.ch_names, len(raw.ch_names))
        
        
        self.raw = raw  # Store the raw data in the instance attribute
        
        return raw

        
    def highlow_pass(self):
        if self.raw is None:
            raise ValueError("Raw data is not loaded. Call read_file() first.")

        # Apply high-pass filter
        self.raw.filter(l_freq=1, h_freq=None, fir_design='firwin', filter_length='auto',
                        l_trans_bandwidth='auto', h_trans_bandwidth='auto', method='fir',
                        phase='zero', fir_window='hamming', verbose=True)

        # Apply low-pass filter if needed
        self.raw.filter(l_freq=None, h_freq=40, fir_design='firwin', filter_length='auto',
                        l_trans_bandwidth='auto', h_trans_bandwidth='auto', method='fir',
                        phase='zero', fir_window='hamming', verbose=True)

        return self.raw
        
        
        
    
    def rename_events(self):
        
        if self.raw is None:
            raise ValueError("Raw data is not loaded. Call read_file() first.")

        min_duration = 1 / self.raw.info['sfreq']
        # Find events and drop unnecessary
        events_ID = mne.find_events(self.raw, stim_channel="Status", initial_event=True, shortest_event=1, min_duration = min_duration)
       # print('These are the events', events_ID)
        events = mne.pick_events(events_ID, exclude=[3, 5, 40, 42, 62])
        #print('These are the events after excluding:', events)


        unsuccessful_event_index = None  # For sanity check

        for i, event in enumerate(events):
            if event[2] == 10:
                next_event_index = i + 1
                while next_event_index < len(events) and events[next_event_index][2] == 61:
                    events[next_event_index][2] = 610  # Modify the trigger value
                    next_event_index += 1
            elif event[2] == 20:
                next_event_index = i + 1
                while next_event_index < len(events) and events[next_event_index][2] == 61:
                    events[next_event_index][2] = 620  # Modify the trigger value
                    next_event_index += 1
            elif event[2] == 30:
                next_event_index = i + 1
                while next_event_index < len(events) and events[next_event_index][2] == 61:
                    events[next_event_index][2] = 630  # Modify the trigger value
                    next_event_index += 1
            else:
                unsuccessful_event_index = i

        # Check if any events with trigger value 61 are still present
        if any(event[2] == 61 for event in events):
            print("Renaming was unsuccessful :(")
            if unsuccessful_event_index is not None:
                print("Problematic event index:", unsuccessful_event_index)
                print("Problematic event details:", events[unsuccessful_event_index])

        # Define epoch parameters
        tmin = -2
        tmax = 3

        
        
        # Rename events with numerical IDs to have corresponding event names
        event_dict = {"10": 10, "20": 20, "30": 30, "selftap": 41, "VP610": 610, "VP620": 620, "VP630": 630}
        #raw_resampled, events_resampled = self.raw.copy().resample(256, npad='auto', events=events)
        #del raw_resampled
        # Epoch signal
        epochs = mne.Epochs(self.raw, events, event_id=event_dict, tmin=tmin, tmax=tmax, baseline=None)
        
        del self.raw                                # Delete raw data after processing
        # Divide the epochs into smaller pieces
        
        epochs.drop_bad()
        num_pieces = 6                           # split epochs into 4 pieces 
        piece_size = len(epochs) // num_pieces
        resampled_epochs_pieces = []

        for idx in range(num_pieces):
            
            start_idx = idx * piece_size
            end_idx = start_idx + piece_size if idx < num_pieces - 1 else len(epochs)
            epochs_piece = epochs[start_idx:end_idx]
            epochs_piece.load_data()
            epochs_rs_piece = epochs_piece.resample(sfreq=256)
            resampled_epochs_pieces.append(epochs_rs_piece)

        # Concatenate the resampled pieces
        self.epochs = mne.concatenate_epochs(resampled_epochs_pieces)
        
    
        #epochs.load_data()
        #Convert to float32   
        # Resample the epochs
        #epochs_rs = epochs.resample(sfreq=256)
        self.epochs.plot(picks= ['eeg'],n_channels=64,n_epochs= 25,block=True)
        print(epochs.drop_log)
        

        return self.epochs



    def plot_comp(self, epochs, picks, participant_id):
        # Call plot_properties to generate the figures
        figs = self.ICA.plot_properties(epochs, picks=picks, log_scale=False)
        
        # Save each figure separately
        for i, fig in enumerate(figs):
            fig.savefig(f'C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Master in AI/Final Thesis/Figures/ICA/ICA_component_{participant_id}_{i}.png')
        
    
    def plotsignal(self, channels, pick_min, pick_max):
        if self.epochs is None:
            print("No epochs data available. Please load epochs data before plotting.")
            return
        ch_names = self.epochs.ch_names
        picks = ch_names[pick_min:pick_max]
        self.epochs.plot(n_channels=channels, picks=picks, events=False, title='EEG Data for the Channels')
        
    
    def drop_bad_epochs(self, bad_epoch_indices=None, p_id=None):
        if self.epochs is None:
            print("No epochs data available. Please run rename_events before dropping bad epochs.")
            return None

        if bad_epoch_indices is not None:
            # Drop bad epochs based on the provided indices
            self.epochs.drop(bad_epoch_indices)
            print("Bad epochs dropped based on provided indices.")
        else:
            # Check if bad epochs were marked interactively during plotting
            if not self.epochs.drop_log:
                print("No bad epochs marked interactively. Nothing to drop.")
            else:
                # Drop bad epochs marked interactively during plotting
                self.epochs.drop()

                print("Bad epochs dropped based on interactive marking.")
                
                
                
        # After we drop bad trials we save the file in a dataframe
        #direc = "C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Thesis Code/Data/epochs/"
       
        #self.epochs.save(direc + p_id + '-epo.fif', fmt='single', overwrite=True, verbose=None)
        
        return self.epochs 
        
        #list = [p_id,self.epochs]
        
    
            
    
    
    
    def fit_ica(self, p_id):
        
        """
        The objective is to remove, eye blinks, heartbeats and muscle artefact.Cleaning the data with ICA
        """
        #epochs.load_data(dtype=np.float64)
        # run fast ICA 
        ica = mne.preprocessing.ICA(
            n_components=30, method="fastica", max_iter="auto", random_state=97)
        
       
        
        self.ICA = ica
        
        ica.fit(self.epochs) # fit on epochs
        
         
        #direc = "C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Thesis Code/Data/Ica_files/"
        #self.ICA.save(direc + p_id + '-ica.fif', overwrite=True)
        # Visualise ICA components
        
        #self.epochs.set_montage(self.montage)

        # plot the raw epoched signal and low and hihgpass
        #mne.viz.set_browser_backend('qt') 
        ica.plot_sources(self.epochs) # plot the raw signal 

        # plot scalp field topographies for 30 components. Here we identify manual artifacts like eyeblinks etc.
       
        ica.plot_components() 
        
        # We use MNE to identify muscle components
        muscle_idx_auto, scores = ica.find_bads_muscle(self.epochs)
        ica.plot_scores(scores, exclude=muscle_idx_auto)
        print(
        f"Automatically found muscle artifact ICA components: {muscle_idx_auto}"
        )
        
        return self.ICA
    
        
    def reject_artefact(self, comps):
        self.ICA.exclude = comps 
        
        reconst_epochs = self.epochs.copy()
        self.ICA.apply(reconst_epochs)
        
        self.epochs = reconst_epochs
        
        return self.epochs
             
    def drop_bad_interp(self, channels_list):

        if self.epochs is None:
            print("No epochs data available. Please run rename_events before interpolating bad channels.")
            return None
            
        # ep_ds = self.epochs.copy() 
            
        if not channels_list:
            # No channels to drop
            print("No channels to drop")
        else: 
            # Highlight bad channels
            #ep_ds.info["bads"].extend(channels_list)
            self.epochs.info["bads"].extend(channels_list)

        #ep_ds.set_montage('standard_1020')

        self.epochs.set_montage('standard_1020')
        
        # ep_interp = ep_ds.interpolate_bads(reset_bads=False, method='MNE', verbose=True)
        self.epochs.interpolate_bads(reset_bads=False, method='spline', verbose=None)   
        # average data 
        # use the average of all channels as reference
        self.clean_epochs, ref_data = mne.set_eeg_reference(self.epochs, ref_channels="average", projection=False)

        return self.clean_epochs

    def drop_bad_interp2(self, channels_list, separate_epochs=None):
        if separate_epochs is None:
            if self.epochs is None:
                print("No epochs data available. Please run rename_events before interpolating bad channels.")
                return None
            else:
                epochs_data = self.epochs
        else:
            epochs_data = separate_epochs
                
        if not channels_list:
            # No channels to drop
            print("No channels to drop")
        else: 
            # Highlight bad channels
            epochs_data.info["bads"].extend(channels_list)

        epochs_data.set_montage('standard_1020')
        
        epochs_data.interpolate_bads(reset_bads=True, method='spline', verbose=None)   
        
        # average data 
        # use the average of all channels as reference
        self.clean_epochs, ref_data = mne.set_eeg_reference(epochs_data, ref_channels="average", projection=False)

        return self.clean_epochs
    
    def fit_imported_ica(self,ica, ep,del_comps=None):
        """
        This function is for when you want to use saved ICA and have already have some components to drop or not.
        """
        ica = ica 
        ica.fit(ep)
        ica.plot_components() 
        ica.plot_sources(ep)
        
        if del_comps is not None:
            ica.exclude = del_comps
            ep.load_data()
            reconst_ep = ica.apply(ep)
            return reconst_ep
        else: 
            
            print("No ICA Components were chosen to be zeroed out. Run function again with ICA Components specified")
            return None


processor = EEGDataProcessor()

Using matplotlib backend: QtAgg


In [4]:
def save_epochs(sub,data_dir,epochs):
    # saves Epoch
    epochs.save(data_dir + sub + '-epo.fif', fmt='single',overwrite=True)
    
    
def save_ica(subject,directionary,ica):
        
        ica.save(directionary + subject+ '-ica.fif', overwrite=True)
        
def make_events(epochs):
    
    
     # separate conditions
    motor_source_event41 = epochs['selftap']
    motor_source_event610 = epochs['VP610']
    motor_source_event620 = epochs['VP620']
    motor_source_event630 = epochs['VP630']
        
        # combine all conditions to one list 
    all_motor_components = [motor_source_event41, motor_source_event610, motor_source_event620, motor_source_event630]
    
    return all_motor_components 

In [3]:
mne.viz.set_browser_backend('qt') 
path = "C:/Users/s184075/Thesis/epochs/subj_39-epo.fif"
processor.load_epochs_from_fif(path)

montage = mne.channels.make_standard_montage("standard_1005") 
ep = processor.epochs

ep.set_montage(montage)
del ep
#processor.fit_ica(p_id=participant_id)
processor.fit_ica(p_id='subj_36')
processor.reject_artefact([1,2,5, 8, 9, 10, 12, 13, 18, 19, 21, 23, 24, 25, 29])

processor.plotsignal(64,1,64)


Using qt as 2D backend.
Reading C:\Users\s184075\Thesis\epochs\subj_39-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =   -2000.00 ...    2996.09 ms
        0 CTF compensation matrices available
Not setting metadata
680 matching events found
No baseline correction applied
0 projection items activated
Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 30 components
Fitting ICA took 27.5s.
Not setting metadata
680 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Automatically found muscle artifact ICA components: [10, 12, 13, 19, 21, 23, 24, 25, 29]
Applying ICA to Epochs instance
    Transforming to ICA space (30 components)
    Zeroing out 15 ICA components
    Projecting back using 64 PCA components


In [4]:
IC = processor.ICA
save_ica('subj_36','C:/Users/s184075/thesis/ICA/',IC)

Overwriting existing file.
Writing ICA solution to C:\Users\s184075\thesis\ICA\subj_36-ica.fif...


In [8]:
processor.drop_bad_interp(['AFz'])
clean_ep = processor.clean_epochs
direc = "C:/Users/s184075/Thesis/clean_epochs/" 
save_epochs('subj_36',direc,clean_ep)


Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.


    Automatic origin fit: head of radius 95.6 mm
Computing interpolation matrix from 55 sensor positions
Interpolating 9 sensors
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Overwriting existing file.
Overwriting existing file.


In [9]:
processor.plotsignal(64,1,64)

In [5]:
mne.viz.set_browser_backend('qt') 
import os 
#del rereferenced_epochs, ica
# Specify 'bad_trials', only true if they have been identified

#path = "C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Thesis Code/Data/Subjects/subj_07_070223_task.bdf"
path = "C:/Users/s184075/thesis/subjects/subj_44_270223_task.bdf"
file_name = os.path.basename(path)  # Get the file name from the path
participant_id = file_name[:7] # get P_Id 
# Create HDF5 file with st


# Call method to read the file
processor.read_file(path)

# Call method for high and low paas 
processor.highlow_pass()

# Call the method to rename events and create epochs
processor.rename_events()



Using qt as 2D backend.
Extracting EDF parameters from C:\Users\s184075\thesis\subjects\subj_44_270223_task.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\s184075\thesis\subjects\subj_44_270223_task.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7755775  =      0.000 ...  3787.000 secs...


C:\Users\s184075\AppData\Local\Temp\ipykernel_7660\4002148146.py:58: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_bdf(path, exclude = channels_to_exclude_str ,  preload=True)
C:\Users\s184075\AppData\Local\Temp\ipykernel_7660\4002148146.py:58: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_bdf(path, exclude = channels_to_exclude_str ,  preload=True)


<Info | 8 non-empty values
 bads: []
 ch_names: A1, A2, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, T7, ...
 chs: 64 EEG, 1 Stimulus
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 417.0 Hz
 meas_date: 2023-02-27 14:30:02 UTC
 nchan: 65
 projs: []
 sfreq: 2048.0 Hz
 subject_info: 1 item (dict)
>
These are the channels and lenght of channnels, ['A1', 'A2', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'Status'] 65
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain de

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.8s


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 677 samples (0.331 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.3s


11016 events found on stim channel Status
Event IDs: [    3     5    10    20    30    40    41    42    61    62 65536]
Not setting metadata
5298 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 5298 events and 10241 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 883 events and 10241 original time points ...
Using data from preloaded Raw for 883 events and 10241 original time points ...
Using data from preloaded Raw for 883 events and 10241 original time points ...
Using data from preloaded Raw for 883 events and 10241 original time points ...
Using data from preloaded Raw for 883 events and 10241 original time points ...
Using data from preloaded Raw for 883 events and 10241 original time points ...
Not setting metadata
5298 matching events found
No baseline correction applied
Dropped 139 epochs: 0, 840, 1368, 2300, 2301, 2302, 2303, 2304, 2497, 2498, 2499, 2500, 2501, 2502, 2503, 2504

Number of events,5159
Events,10: 2920: 2830: 30VP610: 1769VP620: 1494VP630: 1294selftap: 515
Time range,-2.000 – 2.996 s
Baseline,off


In [5]:
bad_epoch_indices = [1, 55, 56, 57, 58, 59, 170, 171, 172, 173, 174, 175, 1205, 1206, 1225, 1226, 1227, 1228, 1373, 1396, 1409, 1410, 1421, 1422, 1423, 1424, 1425, 1490, 1491, 1492, 1493, 1494, 1495, 4613, 4649, 4650, 4651, 4652, 4665, 4666, 4667, 4668, 4669, 4670, 4671, 4672, 4752, 4753, 4754, 4755, 4756, 4757, 4758, 4773, 4774, 4775, 4776, 4777, 4778, 4779, 4780, 4781, 4803, 4807, 4808, 4814, 4815, 8335, 8336, 8337, 8338, 8339, 8340, 8341, 8342, 8343, 8344, 8481, 8505, 8506, 8507, 8508, 8509, 8555, 8556, 8557, 8558, 8559, 8560, 8589, 8590, 8601, 11774, 11775, 11776, 11777, 11778, 11866, 11867, 11868, 11869, 11870, 11871, 11872, 11873, 11884, 11885, 11886, 11887, 11888, 11889, 11890, 11891, 11996, 11997, 11998, 11999, 12000, 12031, 12146, 12147, 12148, 12149, 12150, 12151, 12152, 12153, 12226, 12247, 12252, 12253, 12265, 12266, 12267]
print(len(bad_epoch_indices))

134


In [5]:
## Marked bad channels if not marked.
%matplotlib
mne.viz.set_browser_backend('qt') 

bad_trials = True 
if bad_trials == True:
    
    processor.drop_bad_epochs(bad_epoch_indices=bad_epoch_indices)

   
    
if bad_trials == False:
   
    # plot epochs 
    processor.plotsignal(64,1,64)
# drop bad trials 


Using matplotlib backend: QtAgg
Dropped 107 epochs: 17, 109, 264, 265, 266, 267, 347, 348, 349, 758, 855, 856, 857, 938, 939, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1209, 1210, 1211, 1245, 1246, 1247, 1248, 1261, 1262, 1263, 1264, 1265, 1689, 1732, 1758, 1798, 1808, 1809, 1885, 1886, 1919, 1996, 1997, 1998, 1999, 2000, 2001, 2099, 2100, 2101, 2130, 2188, 2204, 2205, 2206, 2227, 2244, 2245, 2246, 2331, 2332, 2333, 2334, 2335, 2336, 2337, 2338, 2339, 2340, 2341, 2342, 2343, 2535, 2536, 2636, 2637, 2638, 2661, 2662, 2663, 2664, 2754, 2755, 2756, 2757, 2887, 3057, 3058, 3059, 3060, 3080, 3081, 3141, 3142, 3143, 3144, 3292, 3293, 3294, 3298, 3299, 3300, 3301
Bad epochs dropped based on provided indices.


In [6]:

def save_epochs(sub,data_dir,epochs):
    # saves Epoch
    epochs.save(data_dir + sub + '-epo.fif', fmt='single', overwrite=True)
    
    
def save_ica(subject,directionary,ica):
        
        ica.save(directionary + subject+ '-ica.fif', 
        overwrite=True)
        

In [ ]:
del ica
def get_clean(epochs, channel_list):
    epochs.info["bads"].extend(channel_list)

    #ep_ds.set_montage('standard_1020')

    epochs.set_montage('standard_1020')

    # ep_interp = ep_ds.interpolate_bads(reset_bads=False, method='MNE', verbose=True)
    epochs.interpolate_bads(reset_bads=False, method='spline', verbose=None)   
    # average data 
    # use the average of all channels as reference
    clean_epochs, ref_data = mne.set_eeg_reference(epochs, ref_channels="average", projection=False)
    del epochs
    return clean_epochs

epochs = get_clean(epochs,['T7','O1','Fpz','Fp2','TP8','A2','AF8'])
save_epochs(participant_id,'C:/Users/s184075/Thesis/Data/Clean_epochs/',epochs)

In [6]:
mne.viz.set_browser_backend('qt') 
montage = mne.channels.make_standard_montage("standard_1005") 
ep = processor.epochs
#direc = "C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Thesis Code/Data/epochs/" 
#save_epochs(sub=participant_id,data_dir=direc,epochs=ep)
direc = "C:/Users/s184075/thesis/epochs/"
save_epochs(participant_id,direc,ep)
ep.set_montage(montage)
#processor.fit_ica(p_id=participant_id)
processor.fit_ica(p_id=participant_id)

Overwriting existing file.
Overwriting existing file.
Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 30 components
Fitting ICA took 322.5s.
Not setting metadata
5159 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Automatically found muscle artifact ICA components: [10]


Method,fastica
Fit parameters,algorithm=parallelfun=logcoshfun_args=Nonemax_iter=1000
Fit,49 iterations on epochs (6603520 samples)
ICA components,30
Available PCA components,64
Channel types,eeg
ICA components marked for exclusion,—


In [8]:
#del ep, bad_epoch_indices, Subject, file_name
direc = "C:/Users/s184075/Thesis/ICA/"
IC = processor.ICA
IC.plot_properties(ep, picks= [0,6, 10, 20, 21, 22, 23, 26])

save_ica(participant_id,direc,IC)
del IC

    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
1910 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1910 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1910 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1910 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1910 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1910 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1910 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1910 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1910 matching events found
No baseline correction appl

## Marked ICA COMPONENTS

- Subject_1:  ICA000, ICA002,ICA004, ICA017, ICA021, ICA026, ICA027, ICA029.
- Subject_2:  ICA000, ICA001, ICA009, ICA015, ICA017, ICA018, ICA020, ICA021, ICA022, ICA023, ICA025, ICA029.
- Subject_3:  ICA000, ICA004, ICA007, ICA015, ICA016, ICA017, ICA018, ICA019 ICA022,ICA23, ICA029. 
- Subject_4:  ICA000, ICA002, ICA011, ICA015, ICA019, ICA020, ICA022, ICA026, ICA029
- Subject_5:  ICA003, ICA004, ICA005, ICA006, ICA008, ICA011, ICA012, ICA014, ICA015, ICA016, ICA018, ICA019, ICA020, ICA021, ICA022, ICA025, ICA027, ICA028
- Subject_7:  ICA000, ICA003, ICA008, ICA013, ICA016, ICA017, ICA018,  ICA020, ICA021, ICA022, ICA023, ICA024, ICA026, ICA027, ICA028
- Subject_8:  ICA000, ICA002, ICA007, ICA011, ICA012, ICA021, ICa023, ICA024, ICA026, ICA027, ICA028, ICA029
- Subject_10: ICA001, ICA002, ICA013, ICA014, ICA023, ICA025, ICA026, ICA028, ICA029
- Subject_11: ICA001, ICA003, ICA005, ICA011, ICA012, ICA013, ICA015,  ICA016, ICA020, ICA021, ICA022, ICA023, ICA027
- Subject_12: ICA000, ICA003, ICA011, ICA015, ICA025, ICA028, ICA029, 
- Subject_13: ICA002, ICA015, ICA020, ICA022, ICA023, ICA024, ICA025, ICA026, ICA028, ICA029.
- Subject_14: ICA000, ICA002, ICA010, ICA011, ICA012, ICA013, ICA015, ICA017, ICA018, ICA019, ICA022, ICA023, ICA025, ICA026, ICA027, ICA028
- Subject_15: ICA000, ICA004, ICA011, ICA013, ICA014, ICA015, ICA016, ICA017, ICA018, ICA020, ICA021, ICA022, ICA024, ICA025, ICA027, ICA029.
- Subject_16: ICA000, ICA001, ICA003, ICA006, ICA009, ICA010, ICA012, ICA013, ICA014, ICA016, ICA017, ICA018, ICA022, ICA023, ICA024, ICA025, ICA026, ICA027, ICA028, ICA029.
- Subject_17: ICA004, ICA015, ICA019, ICA027.
- Subject_18: ICA000, ICA001, ICA009, ICA010, ICA012, ICA013, ICA014, ICA015, ICA017, ICA019, ICA020, ICA021, ICA022, ICA024, ICA026, ICA029.
- Subject_20: ICA000, ICA005, ICA009, ICA010, ICA011, ICA013, ICA014, ICA015, ICA016, ICA017, ICA018, ICA019, ICA020, ICA021, ICA022, ICA024, ICA025, ICA026, ICA027.
- Subject_21: ICA000, ICA007, ICA016, ICA018, ICA019, ICA020, ICA023, ICA026, ICA028.
- Subject_22: ICA000, ICA001, ICA004, ICA006, ICA007, ICA013, ICA014, ICA017, ICA018, ICA019, ICA025, ICA028
- Subject_23: ICA001, ICA008, ICA010, ICA012, ICA013, ICA014, ICA016, ICA019, ICA022, ICA025, ICA027.
- Subject_24: ICA000, ICA008, ICA018, ICA019, ICA023, ICA025, ICA026, ICA028.
- Subject_25: ICA000, ICA001, ICA003, ICA006, ICA007, ICA009, ICA010, ICA011, ICA013, ICA014, ICA015, ICA019, ICA027.
- Subject_26: ICA000, ICA004, ICA009, ICA016, ICA018, ICA020, ICA021, ICA022, ICA023, ICA024, ICA025, ICA026, ICA029
- subject_27: ICA000, ICA002, ICA009, ICA010, ICA011, ICA012, ICA013, ICA015, ICA016, ICA017, ICA018, ICA020, ICA021, ICA024, ICA025, ICA02
- Subject_28: ICA000, ICA012, ICA014, ICA019, ICA020.
- Subject_29: ICA000, ICA001, ICA002, ICA005, ICA007, ICA008, ICA009, ICA010, ICA013, ICA014, ICA016, ICA018, ICA019, ICA020, ICA021, ICA023, ICA024, ICA025, ICA026, ICA027, ICA028
- Subject_30: ICA000, ICA001, ICA007, ICA008, ICA010, ICA012, ICA014, ICA015, ICA017, ICA022, ICA023, ICA024, ICA027.
- Subject_31: ICA000, ICA010, ICA017, ICA019, ICA020, ICA022, ICA023, ICA024, ICA025, ICA029.
- Subject_32: ICA000, ICA005, ICA012, ICA015, ICA016, ICA017, ICA019, ICA020, ICA021, ICA022, ICA023, ICA027, ICA029
- Subject_33: ICA000, ICA004, ICA012, ICA015, ICA017, ICA018, ICA019, ICA023, ICA024, ICA025, ICA026, ICA028, ICA029
- Subject_34: ICA000, ICA001, ICA003, ICA012, ICA013, ICA014, ICA016, ICA017, ICA020, ICA021, ICA024, ICA025, ICA026, ICA029.
- Subject_35: ICA001, ICA003, ICA005, ICA014, ICA018, ICA021, ICA023, ICA025, ICA027, ICA028.
- Subject_36: ICA000, ICA002, ICA007, ICA009, ICA012, ICA013, ICA014, ICA015, ICA017, ICA018, ICA019, ICA021, ICA022, ICA023, ICA024, ICA028, ICA029.
- Subject_37: ICA000, ICA001, ICA006, ICA009, ICA012, ICA013, ICA017, ICA021, ICA022, ICA027.
- Subject_38: ICA001, ICA003, ICA006, ICA007, ICA009, ICA012, ICA013, ICA017, ICA019, ICA021, ICA022, ICA025, ICA026, ICA029.
- Subject_39: ICA001, ICA002, ICA005, ICA010, ICA012, ICA018, ICA019, ICA021, ICA023, ICA024, ICA025, ICA029.
- Subject_40: ICA000, ICA004, ICA007, ICA013, ICA014, ICA017, ICA021, ICA022, ICA025, ICA027, ICA028, ICA029
- Subject_41: ICA001, ICA004, ICA006, ICA011, ICA013, ICA014, ICA016, ICA019, ICA023
- Subject_42: ICA000, ICA005, ICA008, ICA019, ICA021, ICA022, ICA023, ICA024, ICA026, ICA027, ICA029
- Subject_43: ICA000, ICA003, ICA008, ICA012, ICA015, ICA019, ICA020, ICA021, ICA022, ICA023, ICA024, ICA025, ICA026, ICA027, ICA028, ICA029
- Subject_44:
- Subject_45: ICA000, ICA002, ICA007, ICA009, ICA011, ICA012, ICA013, ICA014, ICA015, ICA016, ICA018, ICA019, ICA020, ICA021, ICA023, ICA024, ICA025, ICA026, ICA027, ICA029
- Subject_46: ICA000, ICA002, ICA010, ICA011, ICA013, ICA014, ICA015, ICA017, ICA018, ICA019, ICA020, ICA023, ICA025, ICA026, ICA027
- Subject_47: ICA000, ICA006, ICA011, ICA017, ICA018, ICA019, ICA023, ICA025, ICA026, ICA028, ICA029
- Subject_48: ICA000, ICA003, ICA011, ICA012, ICA013, ICA015, ICA016, ICA017, ICA018, ICA019, ICA020, ICA021, ICA023, ICA024, ICA025, ICA026, ICA027, ICA028, ICA029.
- subject_49: ICA000, ICA004, ICA006, ICA009, ICA011, ICA012, ICA013, ICA015, ICA016, ICA017, ICA018, ICA019, ICA020, ICA021, ICA022, ICA023, ICA024, ICA025, ICA029.
- Subject_50: ICA001, ICA010, ICA012, ICA017, ICA018, ICA020, ICA021, ICA022, ICA023, ICA027, ICA028, ICA029.

In [8]:
del ep

In [11]:

processor.reject_artefact([0,6, 10, 20, 21, 22, 23, 26])
processor.plotsignal(64,1,64)


MemoryError: Unable to allocate 3.20 GiB for an array with shape (5159, 65, 1280) and data type float64

In [9]:
processor.plotsignal(64,1,64)

In [7]:
processor.drop_bad_interp(['P1'])
clean_ep = processor.clean_epochs

processor.plotsignal(64,1,64)
direc = "C:/Users/s184075/Thesis/clean_epochs/" 
save_epochs(participant_id,direc,clean_ep)

Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.6 mm
Computing interpolation matrix from 63 sensor positions
Interpolating 1 sensors
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
['P1']


### Dataframes

In [26]:
import pandas as pd
import numpy as np

# Define the column names and data types
columns = {
    'P_id': object,
    'Rereferenced_Epochs': float,
    'ICA': object,
    'Alpha_Left_ERD': object,
    'Alpha_Right_ERD': object,
    'Beta_Left_ERD': object,
    'Beta_Right_ERD': object,
}

# Create an empty DataFrame with the defined columns
df = pd.DataFrame(columns=columns.keys())

# Now you have an empty DataFrame with the desired structure
# You can add rows to it as you preprocess each subject
ICA = DP.ICA
# Example of adding data for a subject
subject_data = {
    'P_id': participant_id,
    'Rereferenced_Epochs': rereferenced_epochs,
    'ICA': ICA,  # Placeholder for now
    'Alpha_Left_ERD': None,  # Placeholder for now
    'Alpha_Right_ERD': None,  # Placeholder for now
    'Beta_Left_ERD': None,  # Placeholder for now
    'Beta_Right_ERD': None,  # Placeholder for now
}

# Append the data for the subject to the DataFrame
df = pd.concat([df, pd.DataFrame([subject_data])], ignore_index=True)




# Save in dataframe 


In [15]:
# Later, when more data is available for the same subjec
ica = DP.ICA
subject_manager.add_subject_data(participant_id,ICA=ica)
subject_manager.add_subject_data(participant_id, Alpha_Left_ERD=[ERD_list[0][0].data[1,0,:], ERD_list[1][0].data[1,0,:], ERD_list[2][0].data[1,0,:], ERD_list[3][0].data[1,0,:]])
subject_manager.add_subject_data(participant_id, Alpha_Right_ERD=[ERD_list[0][0].data[0,0,:], ERD_list[1][0].data[0,0,:], ERD_list[2][0].data[0,0,:], ERD_list[3][0].data[0,0,:]])
subject_manager.add_subject_data(participant_id, Beta_Left_ERD=[ERD_list[0][1].data[1,0,:], ERD_list[1][1].data[1,0,:], ERD_list[2][1].data[1,0,:], ERD_list[3][1].data[1,0,:]])
subject_manager.add_subject_data(participant_id, Beta_Right_ERD=[ERD_list[0][1].data[0,0,:], ERD_list[1][1].data[0,0,:], ERD_list[2][1].data[0,0,:], ERD_list[3][1].data[0,0,:]])

In [53]:
# In the cas of no ICA components 
ica = DP.ICA
subject_manager.add_subject_data(participant_id,ICA=ica)
subject_manager.add_subject_data(participant_id, Alpha_Left_ERD=None)
subject_manager.add_subject_data(participant_id, Alpha_Right_ERD=None)
subject_manager.add_subject_data(participant_id, Beta_Left_ERD=None)
subject_manager.add_subject_data(participant_id, Beta_Right_ERD=None)